In [35]:
!pip install pandas numpy tensorflow scikit-learn

In [36]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


In [38]:
# Load dataset, skipping bad lines
dataset_path = '/content/test_dataset.txt'
df = pd.read_csv(dataset_path, sep=' ', header=None, names=['label', 'sentence'], encoding='utf-16', on_bad_lines='skip')

# Display the first few rows of the dataset
print(df.head())

  label sentence
0    මම      යති
0    මම    යත්වා
0    මම     යනවා
0    මම   යනවාලා
0    මම      යනු


In [43]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# Load test data with UTF-16 encoding
test_data_path = '/content/test_dataset.txt'

with open(test_data_path, 'r', encoding='utf-16') as file:
    sentences = [line.strip().split(" ", 1) for line in file.readlines()]

# Process the sentences and load into DataFrame
import pandas as pd

df = pd.DataFrame(sentences, columns=["label", "sentence"])
df["label"] = df["label"].astype(int)

# Check the first few rows of the dataframe to ensure it's loaded correctly
df.head()

# Preprocess data
sentences = df['sentence'].tolist()  # Extract sentences
labels = df['label'].tolist()        # Extract labels (e.g., 0 for incorrect, 1 for correct)

# Convert labels to categorical (if needed)
labels = np.array(labels)

# Tokenize sentences
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(sentences)
sequences = tokenizer.texts_to_sequences(sentences)
word_index = tokenizer.word_index

# Pad sequences
max_seq_length = 50  # Set a max sequence length
padded_sequences = pad_sequences(sequences, maxlen=max_seq_length, padding='post')

# Split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)

# Print shapes
print(f"Training data shape: {X_train.shape}, Testing data shape: {X_test.shape}")
print(f"Training labels shape: {y_train.shape}, Testing labels shape: {y_test.shape}")


Training data shape: (13536, 50), Testing data shape: (3384, 50)
Training labels shape: (13536,), Testing labels shape: (3384,)


In [44]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# Define the RNN model
vocab_size = len(word_index) + 1  # Include OOV token
embedding_dim = 128

model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_seq_length),
    LSTM(128, return_sequences=False),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


423/423 ━━━━━━━━━━━━━━━━━━━━ 52s 117ms/step - accuracy: 0.8470 - loss: 0.4439 - val_accuracy: 0.8629 - val_loss: 0.4073
Epoch 2/5
423/423 ━━━━━━━━━━━━━━━━━━━━ 84s 122ms/step - accuracy: 0.8457 - loss: 0.4359 - val_accuracy: 0.8629 - val_loss: 0.4023
Epoch 3/5
423/423 ━━━━━━━━━━━━━━━━━━━━ 49s 116ms/step - accuracy: 0.8449 - loss: 0.4344 - val_accuracy: 0.8629 - val_loss: 0.4003
Epoch 4/5
423/423 ━━━━━━━━━━━━━━━━━━━━ 81s 115ms/step - accuracy: 0.8494 - loss: 0.4266 - val_accuracy: 0.8629 - val_loss: 0.4047
Epoch 5/5
423/423 ━━━━━━━━━━━━━━━━━━━━ 84s 121ms/step - accuracy: 0.8470 - loss: 0.4321 - val_accuracy: 0.8629 - val_loss: 0.4030


In [45]:
new_sentence = "මම ගෙදර යමු"
new_sequence = tokenizer.texts_to_sequences([new_sentence])
new_padded = pad_sequences(new_sequence, maxlen=max_seq_length, padding='post')
prediction = model.predict(new_padded)
print("Correct" if prediction > 0.5 else "Incorrect")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 327ms/step
Incorrect


In [50]:
new_sentence = "නුබ ගෙදර වේගයෙන් ගියෙහි"
new_sequence = tokenizer.texts_to_sequences([new_sentence])
new_padded = pad_sequences(new_sequence, maxlen=max_seq_length, padding='post')
prediction = model.predict(new_padded)
print("Correct" if prediction > 0.1 else "Incorrect")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Correct


In [54]:
new_sentence = " නුබලා පොත බලා ගෙදර ගියෙහු "
new_sequence = tokenizer.texts_to_sequences([new_sentence])
new_padded = pad_sequences(new_sequence, maxlen=max_seq_length, padding='post')
prediction = model.predict(new_padded)
print("Correct" if prediction > 0.1 else "Incorrect")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Correct


In [55]:
new_sentence = "මම ඔහුගෙන් පොතක් ගත්තෙමි "
new_sequence = tokenizer.texts_to_sequences([new_sentence])
new_padded = pad_sequences(new_sequence, maxlen=max_seq_length, padding='post')
prediction = model.predict(new_padded)
print("Correct" if prediction > 0.1 else "Incorrect")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
Correct


In [57]:
new_sentence = "අපි ගෙදර යමි"
new_sequence = tokenizer.texts_to_sequences([new_sentence])
new_padded = pad_sequences(new_sequence, maxlen=max_seq_length, padding='post')
prediction = model.predict(new_padded)
print("Correct" if prediction > 0.5 else "Incorrect")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Incorrect
